# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle

logging.basicConfig(format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s', datefmt='%H:%M', level=logging.DEBUG)


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [4]:
def calculate_mt(leptons, photons, jets, met):
    # Particles
    if len(leptons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    
    # Transverse mass and Delta
    cos_delta_phi = np.cos(l.phi() - met.phi())
    mt = (2 * l.pt * met.pt * (1. - cos_delta_phi))**0.5
    
    return mt

In [5]:
def calculate_phi(leptons, photons, jets, met, eta_solution=0):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = calculate_mt(leptons, photons, jets, met)
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Select one of them
        if eta_solution > 0:
            eta_v = eta_v_plus
        elif eta_solution < 0:
            eta_v = eta_v_minus
        else:
            dice = np.random.rand()
            if dice > 0.5:
                eta_v = eta_v_plus
            else:
                eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = w + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector)
    l_ = l.boost(vv.boostvector)
    a_ = a.boost(vv.boostvector)
    w_ = w.boost(vv.boostvector)
    r_ = vv # vv.boost(vv.boostvector)

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector.unit()
    x_ = (r_.vector - z_ * r_.vector.dot(z_)).unit()
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l_.vector.dot(x_)
    ly_ = l_.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    
    
def calculate_phi_minus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=-1)
    
    
def calculate_phi_plus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=1)

In [6]:
def setup_observables(delphesprocessor):
    delphesprocessor.reset_observables()
    
    # Default observables (four-momenta of a, l, j, and met)
    delphesprocessor.add_default_observables(
        n_leptons_max=1,
        n_photons_max=1,
        n_jets_max=1,
        include_charge=False,
        include_numbers=False,
        include_met=True
    )
    # Lepton flavour
    delphesprocessor.add_observable('pdgid_l1', 'l[0].pdgid', required=True)

    # Two-particle systems
    delphesprocessor.add_observable('m_la', '(l[0] + a[0]).m', required=True)
    delphesprocessor.add_observable('m_lmet', '(l[0] + met).m', required=True)
    delphesprocessor.add_observable('m_amet', '(a[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_la','(l[0] + a[0]).pt', required=True)
    delphesprocessor.add_observable('pt_lmet','(l[0] + met).pt', required=True)
    delphesprocessor.add_observable('pt_amet','(a[0] + met).pt', required=True)
    delphesprocessor.add_observable('deltaphi_la', 'l[0].phi() - a[0].phi()', required=True)
    delphesprocessor.add_observable('deltaphi_lmet', 'l[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaphi_amet', 'a[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaeta_la', 'l[0].eta - a[0].eta', required=True)

    # Three-particle system
    delphesprocessor.add_observable('m_almet', '(a[0] + l[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_almet', '(a[0] + l[0] + met).pt', required=True)

    # mT(W) and ressurrection phi
    delphesprocessor.add_observable_from_function('mt', calculate_mt, required=True)
    delphesprocessor.add_observable_from_function('phi_minus', calculate_phi_minus, required=True)
    delphesprocessor.add_observable_from_function('phi_plus', calculate_phi_plus, required=True)
    delphesprocessor.add_observable_from_function('phi', calculate_phi, required=True)


In [7]:
def setup_cuts(delphesprocessor, use_tight_cuts=False):
    delphesprocessor.reset_cuts()
    
    if use_tight_cuts:
        delphesprocessor.add_cut('pt_a1 >= 300.')
        delphesprocessor.add_cut('pt_l1 >= 80.')
        delphesprocessor.add_cut('et_miss >= 80.')
        delphesprocessor.add_cut('(deltaphi_la**2 + deltaeta_la**2)**0.5 >= 3.')
        delphesprocessor.add_cut('eta_l1**2 < 2.4**2')
    else:
        delphesprocessor.add_cut('pt_a1 >= 20.')
        delphesprocessor.add_cut('pt_l1 >= 20.')
        delphesprocessor.add_cut('et_miss >= 20.')

## Main loop

In [8]:
n_runs_per_benchmark = 20  # 30  # Number of run_cards

In [14]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=True)
    
    # Load events
    run = i_card + 1
    run_str = str(run)
    if len(run_str) < 2:
        run_str = '0' + run_str
    event_folder = mg_process_dir + 'Events/run_{}'.format(run_str)
    
    dp.add_hepmc_sample(
        event_folder + '/tag_1_pythia8_events.hepmc.gz',
        sampled_from_benchmark=benchmark,
        weights="lhe",  # Workaround for Pythia not propagating weights
        lhe_filename=event_folder + '/unweighted_events.lhe.gz',
    )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_file=log_dir + '/delphes_{}.log'.format(run - 1),
        initial_command='source activate python2',
    )
    
    # Tight analysis
    setup_observables(dp)
    setup_cuts(dp, use_tight_cuts=True)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_tight_{}.h5'.format(i_card))
    
    # Loose analysis
    setup_observables(dp)
    setup_cuts(dp)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


12:59  Starting analysis of runs for card 21
12:59  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
12:59  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_01/tag_1_pythia8_events.hepmc.gz
12:59  Parsing HepMC line: N 1 "0" 

12:59  Found weight labels in HepMC file: ['sm']
12:59  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_01/tag_1_pythia8_events.hepmc.gz
12:59  Unzipping /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_01/tag_1_pythia8_events.hepmc.gz
12:59  Calling source activate python2; /Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/Delphes/DelphesHepMC /Users/johannbrehmer

13:05    First 10 values for observable pt_amet:
[162.20436509 229.17392815 275.75639234 249.29623707 259.45141179
          nan 210.70742018 244.5705838  131.28775514 244.6737765 ]
13:05    First 10 values for observable deltaphi_la:
[-3.27091117 -2.9987402  -2.98728609  3.20378041         nan         nan
 -2.91395426  3.35907865  2.79559761  3.6142211 ]
13:05    First 10 values for observable deltaphi_lmet:
[-0.31742191  0.56251681 -5.79529452  0.16685653         nan  6.22867298
 -0.07020116  0.55241555 -0.18833828  5.91590238]
13:05    First 10 values for observable deltaphi_amet:
[ 2.95348926  3.561257   -2.80800843 -3.03692389  3.59625912         nan
  2.8437531  -2.8066631  -2.98393589  2.30168128]
13:05    First 10 values for observable deltaeta_la:
[-1.74361908 -1.12745023 -0.21395862 -0.2905854          nan         nan
 -0.79858746 -1.70030862  2.49972504  0.27804828]
13:05    First 10 values for observable m_almet:
[ 942.59819351  825.77570086  756.17992925  829.80948474     

13:07  Adding required observable phi_plus defined through external function
13:07  Adding required observable phi defined through external function
13:07  Resetting cuts
13:07  Adding cut pt_a1 >= 20.
13:07  Adding cut pt_l1 >= 20.
13:07  Adding cut et_miss >= 20.
13:07  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_01/tag_1_pythia8_events_delphes.root
13:07  Parsing Delphes file /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_01/tag_1_pythia8_events_delphes.root
13:07  Expected weight labels: ['sm']
13:07  Found 100000 events, 1 weights
13:08    First 10 values for observable et_miss:
[129.50421143  73.28752136  81.88088989  84.66337585  51.94009018
 281.32415771 123.21118164 124.08795929 189.17764282 171.94642639]
13:08    First 10 values for observable phi_miss:
[-2.75772572 -2.09616756  2.97597194  2.50200629 -2.15825772 -3.12964416
 -1.01844561  0.77108508  2

13:12  Applying Delphes-based cuts to LHE weights
13:12  Found weights ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5'] in LHE file
13:12  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_21.h5
13:12  Weight names: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
13:12  HDF5 file does not contain is_nuisance field. Assuming is_nuisance=False for all benchmarks.
13:12  Benchmarks found in HDF5 file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
13:12  Benchmark morphing_basis_vector_1 already in benchmark_names
13:12  Benchmark morphing_basis_ve

13:18    First 10 values for observable pt_a1:
[390.02761841 434.46868896 393.91140747   0.         355.5710144
 412.61416626 368.12063599 377.99154663 358.67263794 371.88342285]
13:19    First 10 values for observable eta_a1:
[ 0.61107421 -0.95004845  1.92603362  0.          0.70303583 -2.41332603
 -1.99152875  0.67025387  2.48164892 -1.49166334]
13:19    First 10 values for observable phi_a1:
[-0.06422035  1.27957165 -2.00266242  0.         -1.27452993 -1.5693543
  0.38788897  2.11575699 -0.07906891 -0.40712723]
13:19    First 10 values for observable e_j1:
[  0.           0.           0.         374.9226462   91.19501496
   0.          72.09391474   0.           0.         204.4078779 ]
13:19    First 10 values for observable pt_j1:
[  0.           0.           0.         306.07116699  32.62981033
   0.          37.98640823   0.           0.          86.74334717]
13:19    First 10 values for observable eta_j1:
[ 0.          0.          0.          0.65877253  1.6850512   0.
  1.2504

...

15:17  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_08/tag_1_pythia8_events.hepmc.gz
15:17  Calling gunzip -k /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_08/tag_1_pythia8_events.hepmc.gz > None
15:18  Parsing HepMC line: N 1 "0" 

15:18  Found weight labels in HepMC file: ['sm']
15:18  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_08/tag_1_pythia8_events.hepmc.gz
15:18  Unzipping /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_08/tag_1_pythia8_events.hepmc.gz
15:18  Calling source activate python2; /Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/Delphes/DelphesHepMC /Users/johannbrehmer/work/projects/madminer/diboson_mining/cards/wgamma/delphes_card.dat

15:37    First 10 values for observable pt_amet:
[          nan  415.47913814 2970.14008001 2214.9499822   320.33577362
 1085.55732597 1394.47063579  116.44112702 2306.49328479 1401.1378333 ]
15:37    First 10 values for observable deltaphi_la:
[        nan -3.04315072         nan -0.99153477 -3.18339771  3.16909114
 -3.14162076         nan -3.12807807 -3.17545795]
15:38    First 10 values for observable deltaphi_lmet:
[ 3.23977518e+00  3.97527218e-02             nan -8.15441608e-02
 -2.18355656e-02  4.30179536e-02  3.18193436e-03             nan
  2.46343315e-02 -6.47201538e-02]
15:38    First 10 values for observable deltaphi_amet:
[        nan  3.08290344  2.93441322  0.90999061  3.16156214 -3.12607318
  3.14480269  3.17886233  3.1527124   3.1107378 ]
15:38    First 10 values for observable deltaeta_la:
[        nan  0.18503091         nan  0.06372941 -0.31479109  0.40919861
  0.85520382         nan  0.69839471 -0.49405497]
15:39    First 10 values for observable m_almet:
[         

15:43  Adding required observable phi_minus defined through external function
15:43  Adding required observable phi_plus defined through external function
15:43  Adding required observable phi defined through external function
15:43  Resetting cuts
15:43  Adding cut pt_a1 >= 20.
15:43  Adding cut pt_l1 >= 20.
15:43  Adding cut et_miss >= 20.
15:43  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_08/tag_1_pythia8_events_delphes.root
15:43  Parsing Delphes file /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_08/tag_1_pythia8_events_delphes.root
15:43  Expected weight labels: ['sm']
15:43  Found 100000 events, 1 weights
15:45    First 10 values for observable et_miss:
[ 128.31944275 1154.87719727  247.69525146 2199.7734375  2283.98071289
 1562.85742188 1477.61462402 2467.0402832   624.59020996 1569.56774902]
15:45    First 10 values for observable phi_miss:
[-0.39171267

15:55    62813 / 100000 events pass everything
15:55  Found weights ['sm'] in Delphes file
15:55  Found 62813 events
15:55  Extracting weights from LHE file
15:55  Found weights ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5'] in LHE file
15:55  Applying Delphes-based cuts to LHE weights
15:55  Found weights ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5'] in LHE file
15:55  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_28.h5
15:55  Weight names: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
15:55  HDF5 file does not contain is_nuisance fiel

RuntimeError: Calling command gunzip -k /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_09/tag_1_pythia8_events.hepmc.gz returned exit code 1.

Std output:

b''Error output:

b"gunzip: can't stat: /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_09/tag_1_pythia8_events.hepmc.gz (/Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma2/Events/run_09/tag_1_pythia8_events.hepmc.gz.gz): No such file or directory\n"

## Combine samples

In [9]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5', k_factors=1./2.)

07:42  
07:42  ------------------------------------------------------------
07:42  |                                                          |
07:42  |  MadMiner v0.1.1                                         |
07:42  |                                                          |
07:42  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
07:42  |                                                          |
07:42  ------------------------------------------------------------
07:42  
07:42  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
07:42  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
07:42  Loading samples 

In [12]:
filenames_in = [sample_dir + 'samples_tight_{}.h5'.format(i_card) for i_card in [2,3,4,5,6,7,8,9,12,13,14,15,16,17,18,19]]

combine_and_shuffle(filenames_in, sample_dir + 'samples_tight.h5', k_factors=1./2.)

07:43  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
07:43  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_2.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight.h5
07:43  Loading samples from file 1 / 16 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_2.h5, multiplying weights with k factor 0.5
07:43  Loading samples from file 2 / 16 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_3.h5, multiplying weights with k factor 0.5
07:43  Loading samples from file 3 / 16 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_4.h5, m